# OpenMP* Device Parallelism (C/C++)

#### Sections
- [Learning Objectives](#Learning-Objectives)
- [Device Parallelism](#Device-Parallelism)
- [GPU Architecture](#GPU-Architecture)
- ["Normal" OpenMP constructs](#"Normal"-OpenMP-constructs)
- [League of Teams](#League-of-Teams)
- [Worksharing with Teams](#Worksharing-with-Teams)
- [Host Device Concurrency](#Host-Device-Concurrency)
- _Code:_ [Lab Exercise: OpenMP Device Parallelism](#Lab-Exercise:-OpenMP-Device-Parallelism)


## Learning Objectives

* Explain basic GPU Architecture 
* Be able to use OpenMP offload worksharing constructs to fully utilize the GPU

### Prerequisites
Basic understanding of OpenMP constructs are assumed for this module. You also should have already went through the  [Introduction to OpenMP Offload module](../intro/intro.ipynb) and [Managing Device Data module](../datatransfer/datatransfer.ipynb), where the basics of using the Jupyter notebooks with the Intel® DevCloud and an introduction to the OpenMP `target` and `target data` constructs were discussed.

***
## Device Parallelism
As we've discussed in the previous modules, the OpenMP `target` construct transfers the control flow to the target device. However, the transfer of control is sequential and synchronous.

In OpenMP, offload and parallelism are separate, so programmers need to explicitly create parallel regions on the target device. In theory, constructs that create parallelism on offload devices can be combined with any OpenMP construct, but in practice, only a subset of OpenMP constructs are useful for the target device.

## GPU Architecture
Before diving into OpenMP parallelism constructs for target divices, let's first examine Intel® GPU architecture.

<img src="Assets/GPU_Arch.png">

Intel® GPUs contain 1 or more slices. Each slice is composed of several Subslices (also called GPU cores). Each subslice contain multiple EUs (likely 8 or more), has it's own thread dispatcher unit, instruction cache, share local memory, and other resources. EUs are compute processors that drive the SIMD ALUs.

The following table displays how the OpenMP concepts of League, Team, Thread, and SIMD are mapped to GPU hardware.

|OpenMP | GPU Hardware |
|:----:|:----|
|SIMD | SIMD Lane (Channel)|
|Thread | SIMD Thread mapped to an EU |
|Team | Group of threads mapped to a Subslice |
|League | Multiple Teams mapped to a GPU |

## "Normal" OpenMP constructs
OpenMP GPU offload support all "normal" OpenMP constructs such as `parallel`, `for`, `barrier`, `sections`, `tasks`, etc. However, not every construct will be useful for the GPU. When using these constructs, the full threading model is only supported with in a subslice, this is because there's no synchronization among subslices, and there's no coherence and memory fence among subslices' L1 caches.

Let's examine the following example.
```c
void saxpy(float a, float* x, float* y, int sz) {
    #pragma omp target map(to:x[0:sz]) map(tofrom(y[0:sz])
    #pragma omp parallel for simd
    for (int i=0; i< sz; i++) {
        y[i] = a * x[i] + y[i];
    }
}
```
Here, we use the `target` pragma to offload the execution to the GPU. We then use `parallel` to create a team of threads, `for` to distribute loop iterations to those threads, and `simd` to request iteration vectorization with SIMD instructions. However, due to the restrictions aforementioned, only one GPU subslice is utilized here, so the GPU would be significantly underutilized. In some cases, the compiler may deduce `team distribute` from `parallel for` and still use the entire GPU.

## League of Teams
To take advantage of multiple subslices, use the `teams` pragma to create multiple **master** threads for execution. When combined with the `parallel` pragma, these master threads become a league of thread teams. Becuase there's no synchronization across teams of threads, the teams could then be assigned to different GPU subslices.

<img src="Assets/teams.JPG">

When using the `teams` construct, the number of teams created is implementation defined. Although, you may optionally specify an upper limit with the **num_teams** clause. The **thread_limit** clause of the `teams` pragma can be optionally used to limit the number of threads in each team.

Example: `#pragma omp teams num_teams(8) thread_limit(16)`

## Worksharing with Teams
After a league of teams is created by `teams`, use the `distribute` construct to distribute chunks of iterations of a loop across the different teams in the league. This is analogous to what the `for` construct does for `parallel` regions. The `distribute` pragma is associated with a loop nest inside a teams region.

For nested loops, the **collapse** clause can be used to specify how many loops are associated with the `distribute` pragma. You may specify a **collapse** clause with a parameter value greater than 1 to collapse associated loops into one large loop.

You can also use **dist_schedule** clause on the `distribute` construct to manually specify the chunk size that are distributed to master threads of each team. For example, `#pragma omp distribute dist_schedule(static, 512)` would create chunks of 512 iterations.

### Example with Combined Constructs
For convenience, OpenMP supports combined constructs for OpenMP offload. The code below shows how a single line can encompass all of the pragmas that we've discussed.
```c
void saxpy (float a, float *x, float *y, int sz) {
    #pragma omp target teams distribute parallel for simd \
                map(to:x(0:sz)) map(tofrom(y(0:sz))
    for (int i=0; i<sz; i++) {
        y[i] = a*x[i] + y[i];
    }
}
```
When these constructs are used without additional clauses, the number of teams created, the number of threads created per team, and how loop iterations are distributed are all implementation defined.
The following diagram breaks down the effects of each pragma in the previous example. Here, we assume that there are a total of 128 loop iterations and that 4 teams, and 4 threads per team are created by the implementation.

1. The `omp target` pragma offloads the execution to device
2. The `omp teams` pragma creates multiple master threads, 4 thread teams in this diagram.
3. The `omp distribute` pragma distributes loop iterations to those 4 thread teams, 32 threads for each team shown.
4. The `omp parallel` pragma creates a team of threads for each master thread (team), 4 threads created for each team shown.
5. The `omp for` pragma distributes the 32 iterations to each of the 4 threads.
6. The `omp simd` pragma specifies that multiple iterations of the loop can be executed using SIMD instructions.

<img src="Assets/distribute.JPG">

## Host Device Concurrency

When a target region is encountered, a host task is generated, which synchronizes the CPU and target device. OpenMP uses tasking to manage execution and dependencies. Add the `nowait` clause so the host does not need to wait for target region to complete.

```c
#pragma omp target nowait
```

Using a `nowait` clause with a `target` construct allows for asynchronous offloading, allowing the host device to continue execution. One way to synchronize a target region back with the host device is by using the `taskwait` construct, which will wait until all tasks complete.

In the following example, the for loop is offloaded to the target device, while the host device continues exectution and performs other work. After both the device and host complete finish, the host device will continue execution. 

```c
#pragma omp target map(to:b,c,d) map(from:a) nowait
{
    #pragma omp teams distribute parallel for simd
    for (i=0; i<500; i++) {
        a[i] = b[i] * c + d;
    }
}

#pragma omp task
    other_work();

#pragma omp taskwait //Synchronization
    a0 = a[0];
```

## Lab Exercise: OpenMP Device Parallelism
In this exercise, we will practice using the offload worksharing constructs on the saxpy function that we've already worked with in the previous modules.

In [1]:
#Optional, see the contents of main.cpp
%pycat main.cpp

//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <omp.h>
#include <stdio.h>

constexpr int ARRAY_SIZE = 256;
constexpr int NUM_BLOCKS = 9;

int main(int argc, char *argv[]) {
  int i, ib, is_cpu = 1, num_teams = 0;
  double tstart, tstop;
  float x[ARRAY_SIZE], y[ARRAY_SIZE];

  float a = 1.0f;
  float tolerance = 0.01f;
  int correct_count = 0;

  // Initialize some data
  for (i = 0; i < ARRAY_SIZE; i++) {
    x[i] = (float)i;
    y[i] = (float)i;
  }

  tstart = omp_get_wtime();

#include "lab/saxpy_func_parallel.cpp"

  tstop = omp_get_wtime();

  printf("Number of OpenMP Devices Available: %d\n", omp_get_num_devices());
  printf("Running on %s.\n", is_cpu ? "CPU" : "GPU");
  printf("Work took %f seconds.\n", tstop - tstart);
  printf("Number of Teams Created: %d\n", num_teams);
  for (int i = 0; i < ARRAY_SIZE; i++)
   

In the cell below, add OpenMP pragmas at the locations indicated to perform the following tasks.
1. For the outer loop, use a **combined** construct to
    1. Create NUM_BLOCKS of **master** threads, use the clause *num_teams(NUM_BLOCKS)*
    2. Distribute the outer loop iterations to the varoius master threads.
2. For the inner loop, use a combined construct to 
    1. Create a team of threads for each master thread.
    2. Distribute inner loop iterations to those threads.
    3. Signal that multiple loop iterations can be executed concurrently with SIMD instructions.

In [2]:
%%writefile lab/saxpy_func_parallel.cpp
#pragma omp target map(from: is_cpu) map(from:num_teams) map(to:x[0:ARRAY_SIZE]) map(tofrom:y[0:ARRAY_SIZE])
{
  // 1. Add pragma to create multiple master threads use clause num_teams(NUM_BLOCKS)
  //    and distribute loop iterations to the various master threads.

  for (ib = 0; ib < ARRAY_SIZE; ib += NUM_BLOCKS) {
    if (ib == 0) {
      // Test if target is the CPU Host or the GPU Device
      is_cpu = omp_is_initial_device();
      // Query number of teams created
      num_teams = omp_get_num_teams();
    }

    // 2. Place the combined pragma here to create a team of threads for each master thread
    //   Distribute iterations to those threads, and vectorize

    for (i = ib; i < ib + NUM_BLOCKS; i++) {
      y[i] = a * x[i] + y[i];
    }
  }
}

Overwriting lab/saxpy_func_parallel.cpp


Next, compile and run the code by using the _run.sh_ script.

In [3]:
#Optionally examine the run script by executing this cell.
%pycat run.sh

#!/bin/bash
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
/bin/echo "##" $(whoami) is running OMP_Offload Module3 -- Parallelism - 1 of 1 main.cpp/main.f90
echo "########## Compiling"
icpx -qopenmp -fopenmp-targets=spir64 main.cpp -o bin/a.out || exit $?
echo "########## Executing"
cd bin
./a.out
echo "########## Done"


Execute the following cell to run the program. Make sure you see the "Passed!" message.

In [1]:
! chmod 755 q; chmod 755 run.sh;if [ -x "$(command -v qsub)" ]; then ./q run.sh; else ./run.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

 If you do not see result in 60 seconds, please restart the Jupyter kernel:
 Kernel -> 'Restart Kernel and Clear All Outputs...' and then try again

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2114894.v-qsvr-1           run_serial.sh    u166450         04:16:34 R batch          
2114895.v-qsvr-1           run_serial.sh    u166450         04:15:16 R batch          
2115004.v-qsvr-1           ...ub-singleuser u166450         00:00:17 R jupyterhub     
2116187.v-qsvr-1           run.sh           u166450                0 Q batch          

Waiting for Output ███████████████████████████ Done⬇

########################################################################
#      Date:           Thu 05 Jan 2023 03:00:58 AM PST
#    Job ID:           2116187.v-qsvr-1.aidevcloud
#      User:           u166450
# Resources:      

_If the Jupyter cells are not responsive or if they error out when you compile the samples, please restart the Kernel and compile the samples again_

Execute the following cell to see the solution.

In [5]:
%pycat saxpy_func_parallel_solution.cpp

#pragma omp target map(from: is_cpu) map(from:num_teams) map(to:x[0:ARRAY_SIZE]) map(tofrom:y[0:ARRAY_SIZE])
{
  // 1. Add pragma to create multiple master threads use clause num_teams(NUM_BLOCKS)
  //    and distribute loop iterations to the various master threads.
#pragma omp teams distribute num_teams(NUM_BLOCKS)
  for (ib = 0; ib < ARRAY_SIZE; ib += NUM_BLOCKS) {
    if (ib == 0) {
      // Test if target is the CPU Host or the GPU Device
      is_cpu = omp_is_initial_device();
      // Query number of teams created
      num_teams = omp_get_num_teams();
    }

    // 2. Place the combined pragma here to create a team of threads for each master thread
    //   Distribute iterations to those threads, and vectorize
#pragma omp parallel for simd
    for (i = ib; i < ib + NUM_BLOCKS; i++) {
      y[i] = a * x[i] + y[i];
    }
  }
}


# Summary
In this module, you have learned the following:
* High-level overview of GPU architecture and how OpenMP constructs map to it.
* Create multiple master threads that can be assigned to GPU subslices using the `teams` construct.
* Distribute loop iterations to those master threads using the `distribute` construct.
* Use the `teams` and `distribute` constructs combined with other OpenMP constructs for better performance.

<html><body><span style="color:green"><h1>Survey</h1></span></body></html>

[Tell us how we did in this module with a short survey. We will use your feedback to improve the quality and impact of these learning materials. Thanks!](https://intel.az1.qualtrics.com/jfe/form/SV_e3yrkDaDE7ZnKmN)

<html><body><span style="color:Red"><h1>Reset Notebook</h1></span></body></html>

##### Should you be experiencing any issues with your notebook or just want to start fresh run the below cell.

In [ ]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
button = widgets.Button(
    description='Reset Notebook',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='This will update this notebook, overwriting any changes.',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
out = widgets.Output()
def on_button_clicked(_):
      # "linking function with output"
      with out:
          # what happens when we press the button
          clear_output()
          !rsync -a --size-only /data/oneapi_workshop/OpenMP_Offload/parallelism/ ~/OpenMP_Offload/parallelism
          print('Notebook reset -- now click reload on browser.')
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])

***

@Intel Corporation | [\*Trademark](https://www.intel.com/content/www/us/en/legal/trademarks.html)